# Evaluación de los modelos

Evaluamos los modelos con las diferentes métricas mencionadas en la práctica: 
| Métrica | Definición |
|---|---|
| F1-score (Fm) | Fm = 2 × (PR × RC) / (PR + RC) |
| Sensibilidad (S) | S = TP / (TP + FN) |
| Exactitud (Acc) | Acc = (TP + TN) / (TP + FP + FN + TN) |
| Especificidad (SP) | SP = TN / (FP + TN) |
| Recall (RC) | RC = TP / (TP + FN) |
| Precisión (PR) | PR = TP / (TP + FP) |
| Tasa de falsos negativos (FNR) | FNR = FN / (TP + FN) |
| Tasa de falsos positivos (FPR) | FPR = FP / (FP + TN) |

Para ello, usamos una función matriz que se encarga de llamar al resto de funciones que calculan las métricas.

In [3]:
import pandas as pd 
import numpy as np

def calculate_statistics(y_true: np.ndarray, y_pred: np.ndarray, name: str) -> None:
    
    # Cálculo de los valores de la matriz de confusión
    TN = np.sum((y_true == 0) & (y_pred == 0))
    TP = np.sum((y_true == 1) & (y_pred == 1))
    FN = np.sum((y_true == 1) & (y_pred == 0))
    FP = np.sum((y_true == 0) & (y_pred == 1))

    print(f"True Negatives: {TN}")
    print(f"True Positives: {TP}")
    print(f"False Negatives: {FN}")
    print(f"False Positives: {FP}")

    
    # Sensibilidad
    S = TP / (TP + FN)
    # Exactitud
    Acc = (TP + TN) / (TP + FP + FN + TN)
    # Especificidad
    SP = TN / (FP + TN)
    # Recall
    RC = TP / (TP + FN)
    # Precisión
    PR = TP / (TP + FP)
    # Tasa de falsos negativos
    FNR = FN / (TP + FN)
    # Tasa de falsos positivos
    FPR = FP / (FP + TN)
    # F1-score
    Fm = 2 * (PR * RC) / (PR + RC)

    # Guardamos los resultados en un DataFrame
    results = pd.DataFrame({
        'Métrica': ['Sensibilidad', 'Exactitud', 'Especificidad', 'Recall', 'Precisión', 'Tasa de Falsos Negativos', 'Tasa de Falsos Positivos', 'F1-score'],
        'Valor': [S, Acc, SP, RC, PR, FNR, FPR, Fm]
    })

    # Guardar los resultados en un archivo CSV
    results.to_csv(f'metricas_validacion_cruzada/{name}.csv', index=False)

    print(results)

calculate_statistics(
    y_true=np.array([0, 1, 1, 0, 1, 0, 1, 1, 0, 0]),
    y_pred=np.array([0, 1, 0, 0, 1, 1, 1, 0, 0, 0]),
    name='ejemplo_resultados'
)

True Negatives: 4
True Positives: 3
False Negatives: 2
False Positives: 1
                    Métrica     Valor
0              Sensibilidad  0.600000
1                 Exactitud  0.700000
2             Especificidad  0.800000
3                    Recall  0.600000
4                 Precisión  0.750000
5  Tasa de Falsos Negativos  0.400000
6  Tasa de Falsos Positivos  0.200000
7                  F1-score  0.666667
